In [1]:
import requests, sys, json
import itertools
import redis
import pandas as pd
from Bio.Blast import NCBIXML
from pprint import pprint
from Bio import SeqIO
from Bio.Blast import NCBIWWW

In [ ]:
###
### NOT PROJECT ###
###

The followiing fetches gene info using NCBI's API:

In [ ]:
# Using NCBI's API to look up a specific gene:
def fetch_endpoint(server, request, content_type):
    r = requests.get(server + request, headers = { "Content-Type" : content_type})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    if content_type == 'application/json':
        return r.json()
    else:
        return r.text

In [ ]:
server = "http://rest.ensembl.org/"
ext = "lookup/id/ENSG00000157764?"
con = "application/json"

In [ ]:
get_gene = fetch_endpoint(server, ext, con)

In [ ]:
pprint (get_gene)

In [ ]:
type(get_gene)

In [ ]:
#######
####### Start of Project 
#######

In [ ]:
# Starting Redis:

In [ ]:
# from the command line:

# connecting to the redis server:
# redis-server /etc/redis/6379.conf

# to connect to the command line interface:
# redis-cli

# check if the server is on:
# 127.0.0.1:6379> PING

# process id of Redis server:
# pgrep redis-server

# to clear out databases in Redis:
# 127.0.0.1:6379> FLUSHDB

# to quit Redis client:
# 127.0.0.1:6379> QUIT

# for more help:
# see https://realpython.com/python-redis/ for Redis -> Python guide

In [ ]:
# Using BBmap to reformat/resize fasta file (also from command line).
# for example:
# reformat.sh in=srr.fasta out=100k.fasta reads=100000

# Connecting to the Redis Client

In [2]:
#r = redis.Redis(db=5) to create database #5
r = redis.Redis(db = 5)

In [ ]:
# practice code
#r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})

In [ ]:
#r.get("Bahamas")

In [ ]:
# working with Velotta lab sequence data:

In [ ]:
# parses fastq file and prints the ID, sequence, and length:
for seq_record in SeqIO.parse("100k.fastq", "fastq"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))
    print('')

The above implies a hierarchy in what the formats can store: FASTA ⊂ FASTQ ⊂ SAM.


In a typical high-throughput analysis workflow, you will encounter all three file types:


- FASTA to store the reference genome/transcriptome that the sequence fragments will be mapped to.
- FASTQ to store the sequence fragments before mapping.
- SAM/BAM to store the sequence fragments after mapping.

for future work, how to work with SAM files:
http://samtools.github.io/hts-specs/SAMv1.pdf

In [ ]:
# Reading in a fastq file using BioPython and saving it as a dictionary:
srr_dict = SeqIO.to_dict(SeqIO.parse("100k.fastq", "fastq"))

In [ ]:
list(srr_dict.keys())[2]

In [ ]:
# Shows all sequence IDs:
#list(srr_dict.keys())

In [ ]:
# printing a specific key's value:
srr_dict['SRR12726684.3']

In [ ]:
#df = pd.DataFrame(srr_dict)

In [ ]:
#df

In [ ]:
#df2 = pd.DataFrame.from_dict(srr_dict, orient='index')

In [ ]:
#df2

In [26]:
# reads-in fastq and assigns all parts of files to lists of related items:
ids = []
lengths = []
names = []
descr = []
dbx = []
la = []
seq = []
with open('10k.fastq') as fasta_file:  # Will close handle cleanly
    for seq_record in SeqIO.parse(fasta_file, 'fastq'):  # (generator)
        ids.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        seq.append(str(seq_record.seq))
        names.append(seq_record.name)
        descr.append(seq_record.description)
        dbx.append(seq_record.dbxrefs)
        la.append(dict(seq_record.letter_annotations))

In [27]:
Qfasta = pd.DataFrame(dict(Name = names, LetterAnn = la)).set_index(['IDs'])

In [28]:
ID_only = Qfasta.Name.str.split('.').str[0]
run_only = Qfasta.Name.str.split('.').str[1]
let_ann = Qfasta.LetterAnn.values

In [29]:
df_total = pd.DataFrame({'ID_only': ID_only.values, 
                    'RunNumber':run_only.values, 
                    'Name': names, 
                    'Full_ID':run_only.index, 
                    'Description': descr, 
                    'Sequence': seq,  
                    'LetterAnn': let_ann, 
                    'length': lengths, 
                    'Dbxrefs': dbx})

In [30]:
# Convert columns to string using map(str) method
#df_total['Sequence'] = df_total["Sequence"].map(str)

In [31]:
#full database
df_total

,ID_only,RunNumber,Name,Full_ID,Description,Sequence,LetterAnn,length,Dbxrefs
0,SRR12726684,1,SRR12726684.1,SRR12726684.1,SRR12726684.1 1 length=101,NCCAGGGCCTCACCAGAGGTCCTGGGTTCATCAATAAATATAGTTA...,"{'phred_quality': [0, 27, 27, 33, 33, 37, 37, ...",101,[]
1,SRR12726684,2,SRR12726684.2,SRR12726684.2,SRR12726684.2 2 length=101,GGCAGGGGTCTAGAGGAGCCTGTTCTATAATCGATAAACCCCGTTA...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]
2,SRR12726684,3,SRR12726684.3,SRR12726684.3,SRR12726684.3 3 length=101,CACAGGGGAAGATGGAGCATACCATAAAAGCTCCAAAGGACGGCAG...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]
3,SRR12726684,4,SRR12726684.4,SRR12726684.4,SRR12726684.4 4 length=101,GGATGGGGAAGCCGGGCGGTGGTGGTGCACGCCTTTAATCCCAGCA...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]
4,SRR12726684,5,SRR12726684.5,SRR12726684.5,SRR12726684.5 5 length=101,ATAAGGGGAACTAGAGAGATATCTCAGCAGTTAAATGCACATATTA...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]
...,...,...,...,...,...,...,...,...,...
9995,SRR12726684,9996,SRR12726684.9996,SRR12726684.9996,SRR12726684.9996 9996 length=101,CTCAGGGCGGGAATATCTGCTTCAGTACAACGACCCCAAACGCACA...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]
9996,SRR12726684,9997,SRR12726684.9997,SRR12726684.9997,SRR12726684.9997 9997 length=101,TCCAGGGGGCATCGAGGCGGACGATGACCGGCTCAACAAGGTCATC...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]
9997,SRR12726684,9998,SRR12726684.9998,SRR12726684.9998,SRR12726684.9998 9998 length=101,CACGGGCGAAGAATTTGTTCACCTCCAAGCATTTCATGAAAAAGCT...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]
9998,SRR12726684,9999,SRR12726684.9999,SRR12726684.9999,SRR12726684.9999 9999 length=101,TTCTGGGGAGATTTACAGGAATCACGTATTAATTTAAAAAAAAAAA...,"{'phred_quality': [33, 33, 33, 33, 33, 37, 37,...",101,[]


In [12]:
#df_noseq = df_total[['ID_only', 'RunNumber','Name','Full_ID','Description']]
#df_noseq.head()

,ID_only,RunNumber,Name,Full_ID,Description
0,SRR12726684,1,SRR12726684.1,SRR12726684.1,SRR12726684.1 1 length=101
1,SRR12726684,2,SRR12726684.2,SRR12726684.2,SRR12726684.2 2 length=101
2,SRR12726684,3,SRR12726684.3,SRR12726684.3,SRR12726684.3 3 length=101
3,SRR12726684,4,SRR12726684.4,SRR12726684.4,SRR12726684.4 4 length=101
4,SRR12726684,5,SRR12726684.5,SRR12726684.5,SRR12726684.5 5 length=101


In [13]:
# for future sorting
#df1 = df.sort_values('score',ascending = False).groupby('pidx').head(2)

In [32]:
# Creating a smaller database for BLAST use
df_seq = df_total[['Full_ID', 'Sequence']]
df_seq_sm = df_seq.head()
df_seq_sm

,Full_ID,Sequence
0,SRR12726684.1,NCCAGGGCCTCACCAGAGGTCCTGGGTTCATCAATAAATATAGTTA...
1,SRR12726684.2,GGCAGGGGTCTAGAGGAGCCTGTTCTATAATCGATAAACCCCGTTA...
2,SRR12726684.3,CACAGGGGAAGATGGAGCATACCATAAAAGCTCCAAAGGACGGCAG...
3,SRR12726684.4,GGATGGGGAAGCCGGGCGGTGGTGGTGCACGCCTTTAATCCCAGCA...
4,SRR12726684.5,ATAAGGGGAACTAGAGAGATATCTCAGCAGTTAAATGCACATATTA...


Parse XML BLAST data into a Record.Blast object.

Parses XML output from BLAST (direct use discouraged). This (now) returns a list of Blast records. Historically it returned a single Blast record. You are expected to use this via the parse or read functions.

In [33]:
new_seq = df_seq_sm['Sequence'].iloc[0]
xd = str(new_seq)
xd

'NCCAGGGCCTCACCAGAGGTCCTGGGTTCATCAATAAATATAGTTACAGTCAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTAAATTGGGAGAAACACACT'

In [34]:
# Working BLAST
#seq = 'NCCAGGGCCTCACCAGAGGTCCTGGGTTCATCAATAAATA'
result_handle = NCBIWWW.qblast("blastn", "nt", xd)
blast_result = open("my_blast.xml", "w")
blast_result.write(result_handle.read())
blast_result.close()
result_handle.close()

In [35]:
E_VALUE_THRESH = 1e-05

In [36]:
# this works! (creates lists of possible matches)
alignx = []
hspx = []
for record in NCBIXML.parse(open("my_blast.xml")):
    if record.alignments : #skip queries with no matches
        #print("QUERY: %s" % record.query[:60])
        for align in record.alignments:
            for hsp in align.hsps:
                if hsp.expect < E_VALUE_THRESH:
                    #print("MATCH: %s " % align.title[:60])
                    #print(hsp.expect)
                    alignx.append(align.title)
                    hspx.append(hsp.expect)

In [37]:
alignx[0]

'gi|224922740|ref|NR_027324.1| Rattus norvegicus H19, imprinted maternally expressed transcript (non-protein coding) (H19), long non-coding RNA'

In [38]:
# gene ID
ID_alignx = alignx[0].split('|')[0] +  alignx[0].split('|')[1]
ID_alignx

'gi224922740'

In [39]:
# Turning xml parse into a data table:

#from https://programtalk.com/vs4/python/4932/phageParser/
def parse_blast(resultfile): #takes in the BLAST result, outputs list that can be made into csv
    result_handle = open(resultfile)
    blast_records = NCBIXML.parse(result_handle)
    csv_list = []
    
    header = [  'Query',
                'Name', 'Length', 'Score', 'Expect',
                'QueryStart', 'QueryEnd',
                'SubjectStart', 'SubjectEnd'
            ]
    
    csv_list.append(header)
    count = 0
    for blast_record in blast_records:
        '''help(blast_record.alignments[0].hsps[0])''' # these give help info for the parts 
        '''help(blast_record.alignments[0])        '''
        count +=1
        
        query = blast_record.query
        for alignment in blast_record.alignments:

            name = alignment.title
           # print('Alignment ID:',alignment.title)
            length = alignment.length
            #print('Alignment length:',alignment.length)
            hsp = alignment.hsps[0] # I don't know if we will ever have more than one, so might as well take the first one.
           # print('hsp score:',hsp.score)
            score = hsp.score
            expect = hsp.expect
            querystart = hsp.query_start
            queryend = hsp.query_end
            subjectstart = hsp.sbjct_start
            subjectend = hsp.sbjct_end
            row = [query,name,length,score,expect,querystart,queryend,subjectstart,subjectend]
            csv_list.append(row)
            print()
            
    result_handle.close()
    return csv_list

In [41]:
xmlparse = parse_blast('my_blast.xml')
xmlparse[2]

['No definition line',
 'gi|169642191|gb|BC160921.1| Rattus norvegicus cDNA clone IMAGE:7461928',
 2103,
 143.0,
 5.35266e-26,
 2,
 80,
 2009,
 2087]

In [42]:
df_xmlparse = pd.DataFrame(xmlparse)
df_xmlparse

,0,1,2,3,4,5,6,7,8
0,Query,Name,Length,Score,Expect,QueryStart,QueryEnd,SubjectStart,SubjectEnd
1,No definition line,gi|224922740|ref|NR_027324.1| Rattus norvegicu...,2369,143.0,0.0,2,80,2275,2353
2,No definition line,gi|169642191|gb|BC160921.1| Rattus norvegicus ...,2103,143.0,0.0,2,80,2009,2087
3,No definition line,gi|51131|emb|X58196.1| M.musculus H19 mRNA,1899,141.0,0.0,2,79,1796,1873
4,No definition line,gi|298889025|emb|FQ223810.1| Rattus norvegicus...,328,135.0,0.0,2,76,254,328
5,No definition line,gi|68534369|gb|BC099104.1| Rattus norvegicus c...,2270,135.0,0.0,2,76,2196,2270
6,No definition line,gi|298888893|emb|FQ223682.1| Rattus norvegicus...,432,131.0,0.0,2,74,360,432
7,No definition line,gi|19264032|gb|BC025150.1| Mus musculus H19 fe...,1171,122.0,0.0,2,67,1106,1171
8,No definition line,gi|1315515462|gb|KY242301.1| Capra hircus H19 ...,6948,102.0,0.0,2,78,6817,6894
9,No definition line,gi|1820964098|ref|XR_004386787.1| PREDICTED: R...,2334,92.0,0.0,2,52,2284,2334


In [ ]:
# try to get this to work, is not currently working

In [ ]:
server = 'http://rest.ensembl.org/'
ext2 = 'lookup/id/1820964098?'
con = 'application/json'

In [ ]:
get_gene2 = fetch_endpoint(server, ext2, con)

In [ ]:
#hspx

In [ ]:
# This works! (shows alignments)
for record in NCBIXML.parse(open("my_blast.xml")):
    for align in record.alignments:
            for hsp in align.hsps:
                print(hsp)
                print()

In [ ]:
#def remove_punctuation(x):
#    try:
 #       x = x.str.replace('[^\w\s]','')
#    except:
#        pass
#    return x

In [ ]:
#df_seq.apply(remove_punctuation)

In [ ]:
# these are the various attributes of a SeqRecord file:
for seq_record in SeqIO.parse("10k.fastq", "fastq"):
    print(seq_record.id)
    print(seq_record.seq)
    print(seq_record.name)
    print(seq_record.description)
    print(seq_record.dbxrefs)
    print(seq_record.letter_annotations)
    print('')

# Parts of a FASTA file

https://biopython.org/docs/1.75/api/Bio.SeqRecord.html

id: Identifier such as a locus tag (string)

seq: The sequence itself (Seq object or similar)

Additional attributes:
name: Sequence name, e.g. gene name (string)

description: Additional text (string)

dbxrefs: List of database cross references (list of strings)

features: Any (sub)features defined (list of SeqFeature objects)

annotations: Further information about the whole sequence (dictionary). Most entries are strings, or lists of strings.

letter_annotations: Per letter/symbol annotation (restricted dictionary). This holds Python sequences (lists, strings or tuples) whose length matches that of the sequence. A typical use would be to hold a list of integers representing sequencing quality scores, or a string representing the secondary structure.

In [ ]:
######

In [ ]:
###### randomness

In [ ]:
first_rec = dict(itertools.islice(srr_dict.items(), 1))

In [ ]:
first_rec

In [ ]:
type(first_rec)

In [ ]:
######

In [ ]:
######

In [ ]:
#viewing the mini dataframe
df_seq_sm

In [ ]:
#Converting the mini dataframe to a dictionary
seq_dict = df_seq_sm.set_index('Full_ID').T.to_dict('list')
seq_dict

In [ ]:
# print all dictionary keys
seq_dict.keys()

In [ ]:
# print only the first dictionary key
list(seq_dict.keys())[0]

In [ ]:
#doesn't work so far
#with r.pipeline() as pipe:
#    for seq_id, seq in seq_dict.items():
#        print(seq_id)
#        print(seq)
        #pipe.hmset(seq_id, seq)
    #pipe.execute()

In [ ]:
# (hset) adds the first key/value pair to the Redis database (this code only adds the first record to the database)
# technically adds values to the Redis hash.
r.hset("Sequence", str(list(seq_dict.keys())[0]), str(list(seq_dict.values())[0]))

In [ ]:
# (hget) grabs sequence for the given key from Redis database
print(r.hget("Sequence", str(list(seq_dict.keys())[0])))

In [ ]:
# prints key value pairs in the seq_dict dictionary (the dictionary to be uploaded to Redis)
for i in seq_dict:
    print(i,'->',seq_dict[i])  

In [ ]:
# adds key, values pairs to redis
for i in seq_dict:
    print(i,'->',seq_dict[i]) 
    r.hset("Sequence", str(i), str(seq_dict[i]))

In [ ]:
# grabs the sequence for given key
print(r.hget("Sequence", "SRR12726684.4"))

In [ ]:
# PySpark stuff:

In [ ]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
df_spark = spark.createDataFrame(df_noseq)
df_spark.head(5)

In [ ]:
df_spark.show()
df_spark.printSchema()

In [ ]:
df_spark.show(1, vertical=True)

In [ ]:
df_spark.write.csv('foo.csv', header=True)
spark.read.csv('foo.csv', header=True).show()

In [ ]:
df_spark.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

In [ ]:
#probably don't need the following code:

In [ ]:
# do not use multiple times!!
blast_record = NCBIXML.read(result_handle)

In [ ]:
E_VALUE_THRESH = 0.04

In [ ]:
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

In [ ]:
for alignment in blast_record.alignments:
    print(alignment)